<a href="https://colab.research.google.com/github/SamudaEdiriweera/ML_Project_In_Chemistry/blob/main/ML_In_CHEM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.model_selection import train_test_split
from google.colab import data_table
import pickle
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error, confusion_matrix

In [ ]:
class Stage_1_2:

    def __init__(self):
        self.ohe = None
        self.One_Hot_value = None
        self.df_encode_toxicity_column = None
        self.x_train = None
        self.x_test = None
        self.y_train = None
        self.y_test = None
        self.acc_y1_predict = None
        self.acc_y2_predict = None
        self.acc_y3_predict = None
        self.acc_y4_predict = None
        self.acc_y5_predict = None
        self.acc_y6_predict = None

    def value_count(self,valu):
        df1[valu].value_counts()

    def add_Othernull(self,inpt):
        if inpt == 'Null' :
            return 'OtherNull'
        else :
            return inpt

    def add_Reagent(self,inpt):
        count1 = df1["Reagent"].value_counts().get(inpt,0)
        if count1 == 1:
            return 'Other'
        else :
            return inpt

    def add_Reactant(self,inpt):
        count2 = df1['Reactant_1'].value_counts().get(inpt,0)
        if count2 == 1:
            return 'Other'
        else :
            return inpt

    def Missing_value(self,df,inpt):
        return df[inpt].fillna(df[inpt].mean(),inplace=True)


    def L_Encoder(self,inpt,df):
        le = LabelEncoder()
        encode_value = le.fit_transform(df[inpt])
        return encode_value

    def One_Hot_Encoder(self,inpt,df):         # Toxicity column is OneHotEncoded
        self.ohe = OneHotEncoder(sparse=False)
        self.One_Hot_value = self.ohe.fit_transform(df[inpt].values.reshape(-1,1))
        return self.One_Hot_value

    def New_column(self,inpt,df):               # add the OneHotEncoded column to df1 dataframe
        self.df_encode_toxicity_column = pd.DataFrame(self.One_Hot_value, columns=self.ohe.get_feature_names_out([inpt]))
        df = pd.concat([df, self.df_encode_toxicity_column], axis=1)
        return df

# End the Encoding of df1 dataframe

class Stage_3(Stage_1_2):

    def Get_dummies(self,x):
        x = pd.get_dummies(x)
        return x

    def Split_DS(self,x,y):

        self.x_train, self.x_test, self.y_train, self.y_test = train_test_split(x,y, test_size=0.2, random_state=0)
        return self.x_train, self.x_test, self.y_train, self.y_test

    def Model_1(self,model):
        model.fit(self.x_train, self.y_train)
        self.y1_predict = model.predict(self.x_test)
        self.acc_y1_predict = accuracy_score(self.y1_predict[:,0 ], self.y_test['Reactant_1_polarity'])
        self.acc_y2_predict = accuracy_score(self.y1_predict[:,1 ], self.y_test['Toxicity'])
        self.acc_y3_predict = accuracy_score(self.y1_predict[:,2 ], self.y_test['Polarity'])
        self.acc_y4_predict = accuracy_score(self.y1_predict[:,3 ], self.y_test['Boiling pooint (Bp)'])
        self.acc_y5_predict = accuracy_score(self.y1_predict[:,4 ], self.y_test['Freezing point'])
        self.acc_y6_predict = accuracy_score(self.y1_predict[:,5 ], self.y_test['Dielectric Constant'])
        self.mse_1 = mean_squared_error(self.y1_predict[:,0 ], self.y_test['Reactant_1_polarity'])
        self.mse_2 = mean_squared_error(self.y1_predict[:,1 ], self.y_test['Toxicity'])
        self.mse_3 = mean_squared_error(self.y1_predict[:,2 ], self.y_test['Polarity'])
        self.mse_4 = mean_squared_error(self.y1_predict[:,3 ], self.y_test['Boiling pooint (Bp)'])
        self.mse_5 = mean_squared_error(self.y1_predict[:,4 ], self.y_test['Freezing point'])
        self.mse_6 = mean_squared_error(self.y1_predict[:,5 ], self.y_test['Dielectric Constant'])

        return self.y1_predict,  self.acc_y1_predict, self.acc_y2_predict, self.acc_y3_predict, self.acc_y4_predict, self.acc_y5_predict,
        self.acc_y6_predict, self.mse_1, self.mse_2, self.mse_3, self.mse_4, self.mse_5, self.mse_6

    def Model_2(self,model):
        model.fit(self.x_train, self.y_train)
        self.yy1_predict = model.predict(self.x_test)
        self.acc_yy1_predict = accuracy_score(self.yy1_predict, self.y_test)
        self.conf_matrix = confusion_matrix(self.yy1_predict, self.y_test)
        return self.yy1_predict, self.acc_yy1_predict, self.conf_matrix

    def Param_for_MOC(self,sv):
        param_grid = {
                        'estimator__C': [0.1, 1, 10],
                        'estimator__kernel': ['rbf', 'linear'],
                        'estimator__gamma': [0.1, 1,'auto'],

                    }
        kf = KFold(n_splits=5, shuffle= True, random_state=0)
        grid_search = GridSearchCV(estimator=sv, param_grid=param_grid, scoring='neg_mean_squared_error',cv=kf, n_jobs=-1  )
        grid_fit = grid_search.fit(self.x_train, self.y_train)
        self.best_params_ofMOC = grid_search.best_params_
        self.best_model_for_MOC = grid_fit.best_estimator_
        self.Overall_Model_1_Accuracy = self.best_model_for_MOC.score(self.x_test, self.y_test)

        return self.best_params_ofMOC, self.best_model_for_MOC, self.Overall_Model_1_Accuracy


    def Param_for_SVC(self,sv):
        param_grid = {
                       'C':[0.1,1,10,100],
                       'kernel':['rbf','linear','poly','sigmoid'],
                     }
        kf = KFold(n_splits=5, shuffle= True, random_state=0)
        grid_search = GridSearchCV(estimator=sv, param_grid=param_grid, cv=kf)
        grid_fit = grid_search.fit(self.x_train, self.y_train)
        self.best_params_ofSVC = grid_search.best_params_
        self.best_model_for_SVC = grid_fit.best_estimator_
        self.Overall_Model_2_Accuracy = self.best_model_for_SVC.score(self.x_test, self.y_test)

        return self.best_params_ofSVC, self.best_model_for_SVC, self.Overall_Model_2_Accuracy

class Stage_4(Stage_3):

    def Model_01_Save(self):

       model_pkl_file = 'Solvent_predict_model_01.pkl'
       with open(model_pkl_file,'wb') as file:
        pickle.dump(self.best_model_for_MOC, file)

    def Model_02_Save(self):

       model_pkl_file = 'Solvent_predict_model_02.pkl'
       with open(model_pkl_file,'wb') as file:
        pickle.dump(self.best_model_for_SVC, file)


In [ ]:
df1 = pd.read_csv("/content/drive/MyDrive/ML in CHEM/sample2.csv", encoding='latin-1')
df1 = df1.drop(columns=['Reactant_2','Reactant_2_polarity'])
df1['Dielectric Constant'] = df1['Dielectric Constant'].astype("int32")
df1['Boiling pooint (Bp)'] = df1['Boiling pooint (Bp)'].astype("int32")
df1['Freezing point'] = df1['Freezing point'].astype("int32")

stage04 = Stage_4()
stage04.Missing_value(df1, 'Reaction Temperature ')
df1['Catalyst'] = df1['Catalyst'].apply(stage04.add_Othernull)
df1['Reagent'] = df1['Reagent'].apply(stage04.add_Othernull)

stage04.value_count('Catalyst')
stage04.value_count('Reaction')
stage04.value_count('Reagent')
stage04.value_count('Reactant_1')

df1['Reagent'] = df1['Reagent'].apply(stage04.add_Reagent)
df1['Reactant_1'] = df1['Reactant_1'].apply(stage04.add_Reactant)

stage04.value_count('Reagent')
stage04.value_count('Reactant_1')

df1['Reactant_1_polarity'] = stage04.L_Encoder("Reactant_1_polarity", df1)
df1['Polarity'] = stage04.L_Encoder("Polarity", df1)
df1['Toxicity'] = stage04.L_Encoder('Toxicity', df1)
df1['Reaction Temperature '] = df1['Reaction Temperature '].astype("int32")

In [ ]:
x = df1[['Reaction','Reaction Temperature ', 'Catalyst','Reagent','Reactant_1']]
y = df1[['Reactant_1_polarity','Toxicity','Polarity','Boiling pooint (Bp)', 'Freezing point','Dielectric Constant']]

In [ ]:
x = stage04.Get_dummies(x)

In [ ]:
stage04.Split_DS(x,y)

In [ ]:
sv = SVC(kernel='linear', C=10, random_state=0)

In [ ]:
multioutput_classifier = MultiOutputClassifier(sv)

In [ ]:
stage04.Model_1(multioutput_classifier)

In [ ]:
print(f"Accuarcy of Reactant_1_polarity  ---> {stage04.acc_y1_predict}")
print(f"Accuarcy of Toxicity             ---> {stage04.acc_y2_predict}")
print(f"Accuarcy of Polarity             ---> {stage04.acc_y3_predict}")
print(f"Accuarcy of Boiling pooint (Bp)  ---> {stage04.acc_y4_predict}")
print(f"Accuarcy of Freezing point       ---> {stage04.acc_y5_predict}")
print(f"Accuarcy of Dielectric Constant  ---> {stage04.acc_y6_predict}")

In [ ]:
print(f"MSE of Reactant_1_polarity  ---> {stage04.mse_1}")
print(f"MSE of Toxicity             ---> {stage04.mse_2}")
print(f"MSE of Polarity             ---> {stage04.mse_3}")
print(f"MSE of Boiling pooint (Bp)  ---> {stage04.mse_4}")
print(f"MSE of Freezing point       ---> {stage04.mse_5}")
print(f"MSE of Dielectric Constant  ---> {stage04.mse_6}")

In [ ]:
stage04.Param_for_MOC(multioutput_classifier)

In [ ]:
print(f"Overall Model 1 Accuracy  -->  {stage04.Overall_Model_1_Accuracy}")

In [ ]:
stage04.best_model_for_MOC.predict(stage04.x_test)

In [ ]:
stage04.y_test

In [ ]:
stage04.Model_01_Save()

In [ ]:
xx = df1[['Reactant_1_polarity','Toxicity','Polarity','Boiling pooint (Bp)', 'Freezing point','Dielectric Constant']]
yy = df1[["Solvent"]]

In [ ]:
xx = stage04.Get_dummies(xx)

In [ ]:
stage04.Split_DS(xx,yy)

In [ ]:
sv = SVC(kernel='linear', C=10, random_state=0)

In [ ]:
stage04.Model_2(sv)

In [ ]:
print(f"Accuarcy of Solvent  ---> {stage04.acc_yy1_predict}")
stage04.Param_for_SVC(sv)
print(f"Overall Model 2 Accuracy  -->  {stage04.Overall_Model_2_Accuracy}")
stage04.best_model_for_SVC.predict(stage04.x_test)
stage04.y_test
stage04.Model_02_Save()

In [ ]:
sns.heatmap(stage04.conf_matrix, annot=True, fmt='d')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.show()

In [ ]:
df_array = stage04.y_test.values

In [ ]:
print(df_array)

In [ ]:
import matplotlib.pyplot as plt
y_test_list = df_array.tolist()
yy1_predict_list = stage04.yy1_predict.tolist()

le = LabelEncoder()
a = le.fit_transform(yy1_predict_list)
b = le.fit_transform(y_test_list)

# Assuming predicted_labels and y_test are your predicted and actual labels
plt.figure(figsize=(10, 6))

# Plotting the test values vs. predicted values
plt.scatter(range(len(b)), b, color='black', label='Actual Values', marker='o')
plt.scatter(range(len(a)), a, color='red', label='Predicted Values', marker='x')

# Plotting the line representing the perfect match between actual and predicted values
plt.plot(range(len(b)), b, color='blue', linewidth=1, linestyle='--', label='Perfect Match Line')

plt.xlabel('Data Point')
plt.ylabel('Class Label')
plt.legend()
plt.title('Actual vs. Predicted Values')
plt.show()